### Cross Validation Task

### 약물 A, B, C, X, Y
##### 다중 분류(Multiclass Classification)
- 의학 연구원으로서 동일한 질병을 앓고 있는 일련의 환자에 대한 데이터를 수집했다.
- 치료 과정 동안 각 환자는 5가지 약물, 즉 약물 A, 약물 B, 약물 c, 약물 x 및 y 중 하나에 반응했다.
-  미래에 동일한 질병을 앓는 환자에게 어떤 약물이 적합할 수 있는지 알아보기 위한 모델을 구축한다.

##### feature
- Age: 환자의 나이
- Sex: 환자의 성별
- BP: 혈압
- Cholesterol: 콜레스테롤 수치
- Na_to_K: 나트륨-칼륨

##### target
- Drug: 의약품, 환자에게 효과가 있었던 약

In [1]:
import pandas as pd

drugs_df = pd.read_csv('./datasets/drugs.csv')
drugs_df

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY
...,...,...,...,...,...,...
195,56,F,LOW,HIGH,11.567,drugC
196,16,M,LOW,HIGH,12.006,drugC
197,52,M,NORMAL,HIGH,9.894,drugX
198,23,M,NORMAL,NORMAL,14.020,drugX


##### 레이블 인코딩

In [2]:
from sklearn.preprocessing import LabelEncoder

drugs_encoder = LabelEncoder()
targets = drugs_encoder.fit_transform(drugs_df['Drug'].tolist())
drugs_df['Drug'] = targets

genders_encoder = LabelEncoder()
genders = genders_encoder.fit_transform(drugs_df['Sex'].tolist())
drugs_df['Sex'] = genders

blood_pressures_encoder = LabelEncoder()
blood_pressures = blood_pressures_encoder.fit_transform(drugs_df['BP'].tolist())
drugs_df['BP'] = blood_pressures

cholesterols_encoder = LabelEncoder()
cholesterols = cholesterols_encoder.fit_transform(drugs_df['Cholesterol'].tolist())
drugs_df['Cholesterol'] = cholesterols

##### GridSearchCV 진행

In [4]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# DecisionTree Classifier 생성
decision_tree_classifier = DecisionTreeClassifier()

# 학습 데이터와 테스트 데이터 세트로 분리
features = drugs_df.iloc[:, :-1]
targets = drugs_df.iloc[:, -1]

X_train , X_test , y_train , y_test = train_test_split(features, targets, test_size=0.2)

parameters = {'max_depth':[2, 3, 4], 'min_samples_split':[12, 14, 16]}

In [5]:
import pandas as pd

grid_decision_tree_classifier = GridSearchCV(decision_tree_classifier, param_grid=parameters, cv=5, refit=True, return_train_score=True)

grid_decision_tree_classifier.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [2, 3, 4],
                         'min_samples_split': [12, 14, 16]},
             return_train_score=True)

In [8]:
# DataFrame으로 변환
scores_df = pd.DataFrame(grid_decision_tree_classifier.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 2, 'min_samples_split': 12}",0.8500,7,0.84375,0.81250,0.84375
1,"{'max_depth': 2, 'min_samples_split': 14}",0.8500,7,0.84375,0.81250,0.84375
2,"{'max_depth': 2, 'min_samples_split': 16}",0.8500,7,0.84375,0.81250,0.84375
3,"{'max_depth': 3, 'min_samples_split': 12}",0.8750,4,0.90625,0.84375,0.90625
4,"{'max_depth': 3, 'min_samples_split': 14}",0.8750,4,0.90625,0.84375,0.90625
5,"{'max_depth': 3, 'min_samples_split': 16}",0.8750,4,0.90625,0.84375,0.90625
6,"{'max_depth': 4, 'min_samples_split': 12}",0.9875,1,1.00000,0.93750,1.00000
7,"{'max_depth': 4, 'min_samples_split': 14}",0.9875,1,1.00000,0.93750,1.00000
8,"{'max_depth': 4, 'min_samples_split': 16}",0.9875,1,1.00000,0.93750,1.00000


In [10]:
print(f'GridSearchCV 최적 파라미터: {grid_decision_tree_classifier.best_params_}')
print(f'GridSearchCV 최고 정확도: {grid_decision_tree_classifier.best_score_}')

prediction = grid_decision_tree_classifier.predict(X_test)
print(f'테스트 데이터 세트 정확도: {accuracy_score(y_test, prediction)}')

# refit 된 객체는 best_estimator_로 가져올 수 있으며,
# 이미 grid_decision_tree객체를 GridSearchCV로 작업하여 생성했기 때문에
# 결과는 똑같이 나온다.
estimator = grid_decision_tree_classifier.best_estimator_
prediction = estimator.predict(X_test)
print(f'테스트 데이터 세트 정확도: {accuracy_score(y_test, prediction)}')

GridSearchCV 최적 파라미터: {'max_depth': 4, 'min_samples_split': 12}
GridSearchCV 최고 정확도: 0.9875
테스트 데이터 세트 정확도: 1.0
테스트 데이터 세트 정확도: 1.0
